In [52]:
import re
import string
from natasha import MorphVocab
from loguru import logger
import numpy as np
import pandas as pd
from razdel import tokenize
from tqdm import tqdm

In [57]:
vectorized_news_df = pd.read_csv('reduced_news.csv', delimiter =',')



In [58]:

vectorized_news_df

,Unnamed: 0,url,title,text,topic,tags,date
0,0,https://lenta.ru/news/2019/12/15/prsm/,Россиянам дали советы по выбору чая,Россиянам дали советы при выборе чая. Рекоменд...,Россия,Общество,2019-12-15
1,1,https://lenta.ru/news/2019/12/15/fb/,В Госдуме назвали японское заявление о Курилах...,Спикер Госдумы Вячеслав Володин назвал угрозой...,Россия,Политика,2019-12-15
2,2,https://lenta.ru/news/2019/12/15/kino/,Украинская ЛГБТ-активистка обвинила ню-фотогра...,Украинская ЛГБТ-активистка Виктория Гуйвик обв...,Культура,Культура,2019-12-15
3,4,https://lenta.ru/news/2019/12/15/anons/,Беглого президента Боливии решили арестовать,Исполняющая обязанности президента Боливии Жан...,Мир,Политика,2019-12-15
4,5,https://lenta.ru/news/2019/12/15/apple/,Свадьба трансгендеров в России попала на видео,В ЗАГСе Казани официально вступила в брак пара...,Россия,Общество,2019-12-15
...,...,...,...,...,...,...,...
423822,496252,https://lenta.ru/news/2024/01/01/braziliya-pri...,Пушилин сообщил о раненых в результате обстрел...,Глава Донецкой народной республики (ДНР) Денис...,Бывший СССР,СВО/Украина,2024-01-01
423823,496253,https://lenta.ru/news/2024/01/01/eks-glava-mid...,В России подняли призывной возраст до 30 лет,С 1 января 2024 года начал действовать закон о...,Россия,Политика,2024-01-01
423824,496254,https://lenta.ru/news/2024/01/01/na-ukraine-vy...,В России изменились условия выдачи материнског...,С 1 января 2024 года материнский капитал будут...,Россия,Политика,2024-01-01
423825,496255,https://lenta.ru/news/2024/01/01/pushilin-soob...,На Украине высказались о возможной смене власти,Директор Украинского института анализа и менед...,Бывший СССР,СВО/Украина,2024-01-01


In [51]:
from time import sleep

for i in tqdm(range(10)):
    sleep(3)

 40%|█████████████████████████████████▏                                                 | 4/10 [00:15<00:22,  3.75s/it]


KeyboardInterrupt: 

In [59]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)
from navec import Navec

# Создание объектов для сегментации и морфологического анализа
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

# Функция для лемматизации текста
def lemmatize_text(text):
    # Морфологический анализ сегментов
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    lemmas = []
    vectors = []
    for token in doc.tokens:
        if token.pos != 'PUNCT':
            # Получение леммы для каждого токена
            token.lemmatize(morph_vocab)
            lemmas.append(token.lemma)
            if token.lemma in emb:
                vector = emb[token.lemma]
                vectors.append(vector)
            else:
                vectors.append(np.zeros(300).astype(float))
    vectors = np.asarray(vectors)
    vectors = vectors.mean(axis=0)
    return vectors
# Пример использования функции для лемматизации текста
vectorized_news_df['vectors'] = [lemmatize_text(text) for text in tqdm(vectorized_news_df['text'])]
vectorized_news_df

100%|████████████████████████████████████████████████████████████████████████| 423827/423827 [4:43:21<00:00, 24.93it/s]


,Unnamed: 0,url,title,text,topic,tags,date,vectors
0,0,https://lenta.ru/news/2019/12/15/prsm/,Россиянам дали советы по выбору чая,Россиянам дали советы при выборе чая. Рекоменд...,Россия,Общество,2019-12-15,"[0.03024490944625548, -0.02288997005517325, 0...."
1,1,https://lenta.ru/news/2019/12/15/fb/,В Госдуме назвали японское заявление о Курилах...,Спикер Госдумы Вячеслав Володин назвал угрозой...,Россия,Политика,2019-12-15,"[-0.05232640347721374, -0.009753774931314735, ..."
2,2,https://lenta.ru/news/2019/12/15/kino/,Украинская ЛГБТ-активистка обвинила ню-фотогра...,Украинская ЛГБТ-активистка Виктория Гуйвик обв...,Культура,Культура,2019-12-15,"[0.050539779765297124, 0.046408075792422086, 0..."
3,4,https://lenta.ru/news/2019/12/15/anons/,Беглого президента Боливии решили арестовать,Исполняющая обязанности президента Боливии Жан...,Мир,Политика,2019-12-15,"[-0.03170237842436672, -0.0020857146435280137,..."
4,5,https://lenta.ru/news/2019/12/15/apple/,Свадьба трансгендеров в России попала на видео,В ЗАГСе Казани официально вступила в брак пара...,Россия,Общество,2019-12-15,"[0.0791580648951684, -0.06350357848515903, -0...."
...,...,...,...,...,...,...,...,...
423822,496252,https://lenta.ru/news/2024/01/01/braziliya-pri...,Пушилин сообщил о раненых в результате обстрел...,Глава Донецкой народной республики (ДНР) Денис...,Бывший СССР,СВО/Украина,2024-01-01,"[-0.22318462050999655, -0.0687082931446382, -0..."
423823,496253,https://lenta.ru/news/2024/01/01/eks-glava-mid...,В России подняли призывной возраст до 30 лет,С 1 января 2024 года начал действовать закон о...,Россия,Политика,2024-01-01,"[-0.07169866711848367, -0.007790409707290642, ..."
423824,496254,https://lenta.ru/news/2024/01/01/na-ukraine-vy...,В России изменились условия выдачи материнског...,С 1 января 2024 года материнский капитал будут...,Россия,Политика,2024-01-01,"[-0.00793672584498312, -0.044217701151705625, ..."
423825,496255,https://lenta.ru/news/2024/01/01/pushilin-soob...,На Украине высказались о возможной смене власти,Директор Украинского института анализа и менед...,Бывший СССР,СВО/Украина,2024-01-01,"[-0.08216488192340841, 0.004817717734517323, 0..."


In [60]:
vectorized_news_df.to_csv('reduced_news_with_vectors.csv', encoding='utf-8')